In [0]:
from pyspark.sql.functions import col, date_trunc, sum as _sum, count as _count, when

silver_path = "/Volumes/workspace/default/raw_data/silver_marketing_events"
gold_path = "/Volumes/workspace/default/raw_data/gold_channel_weekly_kpis"

df_silver = spark.read.format("delta").load(silver_path)

# 1) Weekly aggregation (week bucket)
df_gold = (
    df_silver
    .withColumn("week", date_trunc("week", col("event_time")))
    .groupBy("week", "channel")
    .agg(
        _sum("spend").alias("total_spend"),
        _sum("conversion").alias("total_conversions"),
        _sum("revenue").alias("total_revenue"),
        _count("*").alias("num_events")
    )
    .withColumn(
        "roi",
        when(col("total_spend") > 0, col("total_revenue") / col("total_spend")).otherwise(None)
    )
)

display(df_gold)

# 2) Write Gold Delta
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .save(gold_path)
)

print("✅ Gold saved at:", gold_path)

# 3) Read back / validate
df_gold_read = spark.read.format("delta").load(gold_path)
print("Gold row count:", df_gold_read.count())
display(df_gold_read.orderBy(col("week"), col("channel")))


week,channel,total_spend,total_conversions,total_revenue,num_events,roi
2025-01-06T00:00:00.000Z,social,12.0,1,60.0,1,5.0
2024-12-30T00:00:00.000Z,search,25.5,1,120.0,1,4.705882352941177
2025-01-06T00:00:00.000Z,search,18.0,0,0.0,1,0.0
2025-01-06T00:00:00.000Z,tv,170.0,1,200.0,2,1.1764705882352942
2025-01-06T00:00:00.000Z,email,5.0,2,70.0,2,14.0
2024-12-30T00:00:00.000Z,social,10.0,0,0.0,1,0.0


✅ Gold saved at: /Volumes/workspace/default/raw_data/gold_channel_weekly_kpis
Gold row count: 6


week,channel,total_spend,total_conversions,total_revenue,num_events,roi
2024-12-30T00:00:00.000Z,search,25.5,1,120.0,1,4.705882352941177
2024-12-30T00:00:00.000Z,social,10.0,0,0.0,1,0.0
2025-01-06T00:00:00.000Z,email,5.0,2,70.0,2,14.0
2025-01-06T00:00:00.000Z,search,18.0,0,0.0,1,0.0
2025-01-06T00:00:00.000Z,social,12.0,1,60.0,1,5.0
2025-01-06T00:00:00.000Z,tv,170.0,1,200.0,2,1.1764705882352942
